In [1]:
import pandas as pd
from sklearn.metrics.pairwise        import cosine_similarity
from sklearn.metrics.pairwise        import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df_SteamGames=pd.read_csv('steam_games_cleaned.csv')
df_UserReviews=pd.read_csv('user_reviews_cleaned.csv')

In [3]:
# Se crea un nuevo DataFrame llamado df_games que contiene solo las columnas relevantes
game_columns = ['item_id', 'name']
df_games = df_SteamGames[game_columns]

# Se hace un join entre df_games y df_UserItems utilizando 'item_id' como clave
df_joined = pd.merge(df_games, df_UserReviews[['item_id', 'review']], on='item_id', how='inner')

In [4]:
#Se crea una muestra para el modelo
muestra = df_joined.head(20000)

In [5]:
# Se crea el modelo de machine learning con Scikit-Learn
tfidf = TfidfVectorizer(stop_words='english')
muestra=muestra.fillna("")

tdfid_matrix = tfidf.fit_transform(muestra['review'])
cosine_similarity = linear_kernel( tdfid_matrix, tdfid_matrix)

In [6]:
# Se crea la funcion de recomendación de 5 juegos recomendados para el juego ingresado.
def recomendacion_juego(id: int):
    if id not in muestra['item_id'].values:
        return {'mensaje': 'No existe el id del juego'}
    
    titulo = muestra.loc[muestra['item_id'] == id, 'name'].iloc[0]
    idx = muestra[muestra['name'] == titulo].index[0]
    sim_cosine = list(enumerate(cosine_similarity[idx]))
    sim_scores = sorted(sim_cosine, key=lambda x: x[1], reverse=True)
    sim_ind = [i for i, _ in sim_scores[1:6]]
    sim_juegos = muestra['name'].iloc[sim_ind].values.tolist()
    return {'juegos recomendados': list(sim_juegos)}

In [7]:
# Ejemplo de uso del modelo
print(recomendacion_juego(385770))

{'juegos recomendados': ['Clicker Heroes', 'AdVenture Capitalist', 'Carpe Diem', 'Clicker Heroes', 'Crusaders of the Lost Idols']}


In [8]:
muestra

,item_id,name,review
0,282010,Carmageddon Max Pack,oh where do i start dos
1,70,Half-Life,if you own any half life but not this one that...
2,70,Half-Life,this is one of the greatest games i have ever ...
3,70,Half-Life,possibly the best fps game ever
4,70,Half-Life,do i honestly need to recommend this
...,...,...,...
19995,388080,Borderless Gaming,theres not much to say it works and does what ...
19996,312780,Way of the Samurai 4,this game looks like it would be a lot of fun ...
19997,385980,Scott in Space,great platform game control is very easy and i...
19998,327500,Zenzizenzic,geometric bullet hell has never been so good v...
